# FINGERS COUNT COMPUTER VISION PROJECT

## Descripción del proyecto

Este proyecto se trata del ejercicio final del curso 'Python for Computer Vision with OpenCV and Deep Learning' de José Marcial Portilla en Udemy.

En él vamos a ser capaces de detectar una mano, segmentarla y contar los dedos de la misma que se están mostrando gracias a la librería de Python'OpenCV', muy popular dentro del campo de la visión por ordenador.

**NOTA: No se trata de un tutorial sobre cómo funciona OpenCV y sus diferentes funciones, por lo que no entraré a explicar en detalle lo que hace cada uno de sus métodos utilizados. Se trata de una explicación sobre cómo se ha resuelto el problema enunciado anteriormente haciendo uso de las herramientas que nos ofrece OpenCV.

## Solución

La estrategia a seguir será la siguiente:
- Definiremos una región de interés (ROI, Region Of Interest) en nuestra imagen captada por la cámara.
- Calcularemos el valor medio de los pixeles del background (fondo) de esa ROI para, por ejemplo, los primeros 60 frames del video, para así poder detectar cuando la mano se ha introducido en la ROI.
- Una vez ese valor se ha calculado, podemos introducir la mano en esa ROI para que sea detectada. En esa fase aplicaremos thresholding para ayudarnos a extraer la mano de la ROI.
- Una vez la mano esté dentro de la ROI, usaremos la écnica 'Convex Hull' para dibujar un polígono alrededor de la mano.
- Calcularemos el centro (aproximado) de la mano como la intersección entre los puntos más extremos del polígono.
- Dibujaremos un circulo de radio algo menor (parámetro que tendremos que jugar con él para afinar el código) a la distancia entre el centro de la mano y el vértice del polígono  más alejado de éste.
- Posteriormente detectaremos los contornos que están fuera e ese circulo, que corresponderan a los contornos de los dedos.
- Contando el nº de contornos podremos saber el número de dedos que están alejados de ese circulo y por lo tanto extendidos.

### Definición de variables globales

En primer lugar, además de importar las librerias que usaremos en el código, definiremos las variables globales del problema.

Estas son la región de interés (ROI, Region Of Interest) de nuestra imagen, donde introduciremos la mano para que sea detectada, y el fondo de la ROI (background) que al iniciar nuestro programa nos servirá para calcular el valor medio de los pixels de la ROI para poder detectar la mano una vez sea introducida. Al introducir la mano en la ROI el valor de los pixeles cambiará y sabremos que un objeto ha entrado.

In [ ]:
#Importamos las librerias que vamos a usar
import cv2
import numpy as np
from sklearn.metrics import pairwise #Para calcular distancias

In [ ]:
#VARIABLES GLOBALES
background = None  #Se irá actualizando su valor
accumulated_weight = 0.5

#Definimos la región de interés (ROI, Region Of Interest) donde introduciremos la mano para que sea detectada
roi_top = 20
roi_bottom =300
roi_right = 300
roi_left = 600

### Función para calcular el valor medio de los pixeles del background de la ROI

A continuación el código de la función que nos calculará la media del valor de los pixeles del background:

<img src="images\ROI_1.png"> <img src="images\ROI_2.png"> 

In [ ]:
#FUNCIÓN QUE NOS DARÁ LA MEDIA DEL VALOR DE FONDO (AVERAGE BACKGROUND VALUE)
def calc_accum_avg(frame, accumulated_weight):
    global background

    if background is None:
        background = frame.copy().astype('float')
        return None
    
    #Actualizar la variable global 'background' con el 'accumulated_weight'
    cv2.accumulateWeighted(frame, background, accumulated_weight)

### Función para extraer la mano en la ROI

Función que nos extraerá la mano de la ROI, haciendo uso del método 'threshold' y 'findContours' de 'OpenCV'. El threshold lo que hace es cambiar el color de los pixels de la imagen que le pases a negro o blanco según un umbral que se establezca (otro parámetro con el que habría que 'jugar' para afinar el código). Si previamente calculamos la diferencia de los pixels entre el background y un nuevo frame, aquellos valores de pixel que no estén cercanos a cero querrán decir que pertenecen a la mano introducida.

Con 'findContours' lo que hacemos es detectar los contornos de la mano con el threshold aplicado.

<img src="images\THRESHOLD.png"> 

In [ ]:
#FUNCIÓN QUE, A TRAVÉS DEL THRESHOLDING, NOS AYUDARÁ A EXTRAER LA MANO EN LA ROI
def segment(frame, threshold_min = 25):

    global background
    
    #Calculamos la diferencia en valor absoluto entre el fondo (background) y el frame que se le pase a la
    #función
    diff = cv2.absdiff(background.astype('uint8'), frame)

    #Aplicamos un threshold a la imagen asi podemos extraer el primer plano (foreground)
    ret, thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)

    #Extraemos los contornos exteriores de la imagen con el threshold aplicado
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #Si la lista contours está vacía quiere decir que no hemos extraído ningun contorno
    if len(contours) == 0:
        return None
    
    else:
        #Si extraemos algún contorno, el contorno externo más grande deberá de ser la mano (asumiendo
        # que no se va a introducir un objeto con un contorno más grande que la mano en la ROI)
        hand_segment = max(contours, key = cv2.contourArea)

        return thresholded, hand_segment

### Función para contar el nº de dedos

Hasta ahora hemos calculado el contorno exterior de la mano. Ahora, usando ese contorno exterior, vamos a ver cómo calcular los dedos para poder contar cuántos están mostrándose.

Utilizaremos lo que se conoce como 'Convex Hull' para dibujar un polígono alrededor de la mano.
Calcularemos el centro de la mano como la intersección entre los vértices más extremos del polígono (más abajo, más arriba, más a la derecha y más a la izquierda).

Posteriormente calcularemos la distancia entre el centro y esos puntos más extremos para ver cuál es la mayor distancia. Una vez calculada dibujaremos un circulo centrado en el centro de la mano con radio algo menor a la máxima distancia. Los dedos cuyos extremos estén fuera de ese circulo o suficientemente lejos del punto más bajo del polígono serán considerados como dedos extendidos.

<img src="images\hand_convex.png"> <img src="images\DISTANCE.png" alt="Drawing" style="width: 300px;"/> <img src="images\circle.png" alt="Drawing" style="width: 295px;"/>

In [ ]:
def count_fingers(thresholded, hand_segment):

    conv_hull = cv2.convexHull(hand_segment)

    #Punto más elevado del polígono
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    #Punto más bajo del polígono
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    #Punto más a la izquierda del polígono
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    #Punto más a la derecha del polígono
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])

    #Calculamos el centro de la mano
    cX = (left[0] + right[0])//2
    cY = (top[1] + bottom[1])//2

    #Calculamos la distancia euclídea desde el centro de la mano a los puntos extremos del polígono
    distance = pairwise.euclidean_distances([(cX,cY)], Y=[left, right, top, bottom])[0]

    #Cálculo de la distancia máxima
    max_distance = distance.max()

    #Definimos el círculo centrado en el centro de la mano
    radius = int(0.7*max_distance)
    circumference = (2*np.pi*radius)

    #Definimos región de interés del circulo
    circular_roi = np.zeros(thresholded.shape[:2], dtype='uint8')

    #Dibujamos el circulo
    cv2.circle(circular_roi, (cX,cY), radius, 255, 10)

    #A la imagen con el threshold aplicado le 'cortamos' la ROI circular
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    #Extraemos los contornos a la nueva imagen de la región de interés (ROI) del circulo
    contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    #El contador de dedos empezará en 0
    count = 0

    #Hacemos un loop sobre los contornos extraídos de la ROI del circulo para ver si contamos algún dedo más
    for cnt in contours:
        
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        #Para contar un dedo se deben de cumplir las siguientes dos condiciones:

        #1) La región del contorno encontrado no se trata de la parte inferior de la muñeca
        out_of_wrist = (cY +  (cY*0.25)) > (y + h)

        #2) El número de puntos a lo largo del contorno no excede el 15% de la circunferencia del ROI circular (de lo contrario estamos contando puntos fuera de la mano)
        limit_points = ((circumference*0.15) > cnt.shape[0])

        if out_of_wrist and limit_points:
            count +=1

    return count

### Código final

Una vez definidas las funciones y variables que usaremos en el problema pasamos a construir el bloque que ensamblará todo lo anterior y nos permitirá contar los dedos de la mano:

<img src="images\video.png" alt="Drawing" style="width: 300px;"/>

In [ ]:
#PROGRAMA FINAL

#Activamos nuestra cámara
cam = cv2.VideoCapture(0)

#Inicializamos nº de frames a 0
num_frames = 0

#Para guardar el video
#width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
#height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
#writer = cv2.VideoWriter('Finger Count Video.mp4', cv2.VideoWriter_fourcc(*'DIVX'),25, (width, height))

#Grabamos video hasta que sea interrumpido
while True:

    #Obtenemos el frame actual
    ret, frame = cam.read()

    #Le damos la vuelta al frame para que no tenga efecto espejo
    frame = cv2.flip(frame, 1)
    
    #Clonamos el frame
    frame_copy = frame.copy()

    #Extraemos la región de interés definida al principio del proyecto
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    #Convertimos la región de interés a blanco y negro y la emborronamos un poco,
    #ya que asi se suelen conseguir mejores resultados
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7,7), 0)

    #Para los primeros 60 frames calcularemos la media del valor de los pixels del background y mostraremos en el video cuando se está haciendo este cálculo
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)

        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Finger Count', frame_copy)

    else:

        #Ahora que tenemos el fondo podemos extraer la mano
        hand = segment(gray)

        #Hay que checkear si se ha detectado la mano
        if hand is not None:

            thresholded, hand_segment = hand

            #Dibujamos contornos alrededor de la mano en vivo
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0), 1)

            #Contamos los dedos
            fingers = count_fingers(thresholded, hand_segment)

            #Mostramos el resultado
            cv2.putText(frame_copy, str(fingers), (70,45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            #Mostramos la imagen con el umbral (thresholded)
            cv2.imshow('Thresholded', thresholded)
        
    #Dibujamos el rectángulo de la región de interés
    cv2.rectangle(frame_copy, (roi_left, roi_top),(roi_right, roi_bottom), (0,0,255), 5)

    #Incrementamos el número de frames para hacer seguimiento
    num_frames += 1

    #Grabamos el video
    #writer.write(frame_copy)

    #Mostramos la imagen con la mano segmentada
    cv2.imshow('Finger Count', frame_copy)

    #Cerramos la visualización al presionar Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

#Dejamos de usar la cámara y cerramos todas las ventanas
cam.release()
writer.release()
cv2.destroyAllWindows()

### Conclusiones

Como habéis podido observar, con pocas líneas de código y haciendo uso de 'OpenCV' se pueden hacer proyectos bastante interesantes y atractivos.

Si queréis probar el código seguramente tendréis que modificar ciertos parámetros para que se ajuste a la imagen que graba vuestra cámara. Los parámetros más importantes son:
- El valor del umbral mínimo (threshold_min) que se introduce a la función 'segment' para decidir cuando un pixel pasa a negro o blanco. Esto dependerá mucho de la ilumicación de vuestra habitación y de la cantidad de objetos que estén apareciendo en pantalla en la ROI.
- El radio del circulo centrado en vuestra mano. Dependerá de como de grande o chica sea vuestra mano. Si es chica es aconsejable un valor más bajo.
- El número de puntos sobre el contorno de los dedos detectados que no exceda el 15% del perímetro de la ROI del círculo, para asegurarnos que no estamos contando puntos uera de la mano. Igual que antes, dependerá de las dimensiones de vuestra mano.
- El nº de frames que debéis de dejar al principio para que calculer el valor medio de los pixels del background de la ROI. Si tenéis un background con mucho movimiento de objetos o cambios bruscos de iluminación es mejor poner más frames.
- Por último, elegir otros métodos y parámetros de 'OpenCV'. Esta librería tiene muchisimas opciones que pueden ayudarnos a resolver un mismo problema, por lo que existe infinidad de combinaciones de técnicas y métodos.

Espero que hayáis disfrutado con la lectura y qe os haya gustado el proyecto :)